In [72]:
import pandas as pd
from datetime import timedelta
import os
import numpy as np
#from geopy.distance import geodesic
#from io import StringIO
#import math
#import matplotlib.pyplot as plt
#import folium
#import matplotlib.patches as mpatches  # For custom legends
#import ast
#from scipy.spatial import cKDTree
import random  # For mapping a random sample of trips

# Library created to create trips
import importlib
import stph_trips
importlib.reload(stph_trips)

<module 'stph_trips' from '/Users/pam/Desktop/venv/STPH Trip Segmentation (Final)/stph_trips/__init__.py'>

---

*Example of how to inspect a function from a library:*

In [73]:
import inspect

print(inspect.getsource(stph_trips.trip_segmentation))  # View source code 

def trip_segmentation(vehicle_feeds_df, routes_dict, my_dist_cutoff, zero_cutoff, my_dist_threshold, my_time_threshold):
    
    # Step 1
    vehicle_feeds = vehicle_feeds_df.copy()
    vehicle_feeds_engineOn = vehicle_feeds[vehicle_feeds['engineRpm'] > 0].reset_index(drop=True)
    vehicle_feeds_engineOn['identifier'] = ''   ## Initialize an empty "identifier" column (similar to a trip_id)
    vehicle_feeds_engineOn = vehicle_feeds_engineOn.sort_values(by = ['timestamp'])   ## Ensure data is in chronological order

    ################################## ------------ OUTBOUND TRIPS ------------ ##################################
    
    # Step 2
    outbound = nearest_stop_checker(vehicle_feeds_df = vehicle_feeds_engineOn,
                                    routes_dict = routes_dict, 
                                    trip_type = 'outbound', dist_cutoff = my_dist_cutoff)

    # Step 3
    outbound_trips = sequence_checker(outbound, trip_type = 'outbound')

    # Step 4
    outboun

<hr style="border:2px solid darkblue"> 

## Import telematics data

In [2]:
def import_and_concatenate_csvs(folder_path):
    all_dfs = []
    
    # Loop through all files in the folder
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):  # Check if the file is a CSV
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)  # Read CSV into DataFrame
            all_dfs.append(df)  # Store DataFrame in list
    
    # Concatenate all DataFrames into one
    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        return final_df
    else:
        print("No CSV files found in the specified folder.")
        return None

folder_path = "/Users/pam/Desktop/venv/STPH Data Audit/Telematics CSVs" 
telematics_data = import_and_concatenate_csvs(folder_path)

In [3]:
# Data cleaning of telematics
imeis = ['350612076055969', '350612079013932', '350612076078946',
         '350612076068145', '350612076064508', '350544507513318']
routes = ['Route 9', 'Route 9', 'Route 17', 'Route 17', 'Route 9', 'Route 17']

list_of_ejeepneys = pd.DataFrame(data = {'imei': imeis, 'route': routes})

In [4]:
# Fix timestamp
telematics_data['timestamp'] = telematics_data['timestamp'].apply(str).str.replace('+00:00', '').str.replace('T', ' ').str[:19]
telematics_data['timestamp'] = pd.to_datetime(telematics_data['timestamp'], format = '%Y-%m-%d %H:%M:%S')

# Convert imei and deviceCode to string
telematics_data['deviceCode'] = telematics_data['deviceCode'].apply(str).str.replace('.0', '')
telematics_data['imei'] = telematics_data['imei'].apply(str).str.replace('.0', '')

telematics_data = telematics_data.sort_values(by = ['timestamp', 'deviceCode'])

In [5]:
# Add route
telematics_with_route = telematics_data.merge(list_of_ejeepneys, how='left', on = 'imei')
telematics_with_route.dropna(subset = ['route'], inplace = True)
telematics_with_route.reset_index(drop=True, inplace=True)

del telematics_data   ## To save memory 

In [6]:
# Add distance travelled, per vehicle
from stph_trips.trip_summary import haversine  

## Sort by deviceCode and timestamp to ensure correct ordering
telematics_with_route = telematics_with_route.sort_values(['deviceCode', 'timestamp'])

## Compute previous latitude, longitude, and timestamp per device
telematics_with_route['prev_latitude'] = telematics_with_route.groupby('deviceCode')['latitude'].shift(1)
telematics_with_route['prev_longitude'] = telematics_with_route.groupby('deviceCode')['longitude'].shift(1)
telematics_with_route['prev_timestamp'] = telematics_with_route.groupby('deviceCode')['timestamp'].shift(1)

## Compute time difference in seconds
telematics_with_route['time_diff'] = (telematics_with_route['timestamp'] - telematics_with_route['prev_timestamp']).dt.total_seconds()

## Compute distance traveled
telematics_with_route['distanceTravelled'] = telematics_with_route.apply(
    lambda x: haversine(x['prev_latitude'], x['prev_longitude'], x['latitude'], x['longitude']) 
    if pd.notna(x['prev_latitude']) and x['time_diff'] <= 600 else 0,  # Ignore if time_diff > 600 sec (10 min)
    axis=1)
telematics_with_route = telematics_with_route.drop(columns=['prev_latitude', 'prev_longitude', 'prev_timestamp', 'time_diff'])

In [7]:
telematics_with_route = telematics_with_route.reset_index(drop = True)
telematics_with_route.head(5)

,imei,latitude,longitude,timestamp,accelerationInMetersPerSecondSquared,distanceTravelledInKm,distanceTravelledSinceFaultCodesOrClearedInKm,tripOdomenterInKm,vehicleSpeedInKph,boardingCount,...,fuelPressure,fuelRailTemperatureIntake,manifoldAbsolutePressure,massAirFlowRate,throttlePosition,motorCurrentInXxx,averageFuelUse,directFuelRailPressure,route,distanceTravelled
0,350544507513318,10.689347,122.525690,2025-01-10 06:30:32,0,0,-25543,0,0,0,...,3766,0,0,1994,7,0,0,3766,Route 17,0.000000
1,350544507513318,10.689348,122.525693,2025-01-10 06:30:33,0,0,-25543,0,0,0,...,3733,0,0,1988,7,0,0,3733,Route 17,0.000407
2,350544507513318,10.689348,122.525698,2025-01-10 06:30:34,0,0,-25543,0,0,0,...,3798,0,0,1983,7,0,0,3766,Route 17,0.000546
3,350544507513318,10.689352,122.525705,2025-01-10 06:30:35,0,0,-25543,0,0,0,...,3766,0,0,1991,7,0,0,3766,Route 17,0.000809
4,350544507513318,10.689350,122.525712,2025-01-10 06:30:36,0,0,-25543,0,0,0,...,3766,0,0,2013,7,0,0,3766,Route 17,0.000753


<hr style="border:2px solid darkblue"> 

## Trip segmentation

In [70]:
importlib.reload(stph_trips.trip_segmentor)

<module 'stph_trips.trip_segmentor' from '/Users/pam/Desktop/venv/STPH Trip Segmentation (Final)/stph_trips/trip_segmentor.py'>

In [74]:
# Produce the route_dict first, the object name `routes_dict` shall not be changed
routes_dict = stph_trips.obtain_route_dict(path_of_gtfs_shapefiles = '/Users/pam/Desktop/venv/STPH Trip Summary and Segmentation/Route Stops/Shapes/',
                                           min_dist = 100, max_dist = 150)

In [75]:
vehicleFeeds_with_tripID_list = []

for vehicle in telematics_with_route['imei'].unique().tolist():
    print("Now processing: " + vehicle)
    my_df = telematics_with_route[telematics_with_route['imei'] == vehicle].reset_index(drop=True)
    vehicle_feeds_with_tripID = stph_trips.trip_segmentation(vehicle_feeds_df = my_df,
                                                             routes_dict = routes_dict,
                                                             my_dist_cutoff = 100,  ## Increase, default is 50
                                                             zero_cutoff = 60,   ## idle time no longer than 1 min (60 secs)
                                                             my_dist_threshold = 0.3,
                                                             my_time_threshold = 15)   ## trips no shorter than 15 minutes
    vehicleFeeds_with_tripID_list.append(vehicle_feeds_with_tripID)

all_vehicleFeeds_with_tripID = pd.concat(vehicleFeeds_with_tripID_list, ignore_index = True)

Now processing: 350544507513318
Now processing: 350612076055969
Now processing: 350612076064508
Now processing: 350612076068145
Now processing: 350612076078946
Now processing: 350612079013932


In [76]:
all_vehicleFeeds_with_tripID.to_csv("Vehicle feeds with Trip ID - Dec to Jan - GTFS amended 03112025.csv", index = False)

Benchmarking: **14 minutes**

* 2024-12-02 08:31:59
* 2025-01-23 19:02:42

Benchmarking: **21 minutes, without the `trip_super_summary()`**

<hr style="border:2px solid darkblue"> 

## Mapping

In [80]:
# Optional
del telematics_with_route
all_vehicleFeeds_with_tripID['tripID'] = all_vehicleFeeds_with_tripID['imei'] + "_" + all_vehicleFeeds_with_tripID['trip_identifier']
all_vehicleFeeds_with_tripID.head(2)

,imei,latitude,longitude,timestamp,accelerationInMetersPerSecondSquared,distanceTravelledInKm,distanceTravelledSinceFaultCodesOrClearedInKm,tripOdomenterInKm,vehicleSpeedInKph,boardingCount,...,manifoldAbsolutePressure,massAirFlowRate,throttlePosition,motorCurrentInXxx,averageFuelUse,directFuelRailPressure,route,distanceTravelled,trip_identifier,tripID
0,350544507513318,10.688830,122.526290,2025-01-10 06:57:44,0,0,-25543,101,14,0,...,0,2836,10,0,0,4710,Route 17,0.000000,inbound_cuttrip_20250110_065744,350544507513318_inbound_cuttrip_20250110_065744
1,350544507513318,10.688807,122.526387,2025-01-10 06:57:45,0,0,-25543,106,12,0,...,0,2563,10,0,0,4320,Route 17,0.010871,inbound_cuttrip_20250110_065744,350544507513318_inbound_cuttrip_20250110_065744


In [81]:
all_complete_trips = [trip for trip in all_vehicleFeeds_with_tripID['tripID'].unique().tolist() if 'cuttrip' not in trip]
sample_trips = random.sample(all_complete_trips, 10)
for trip in sample_trips:
    feeds = all_vehicleFeeds_with_tripID[all_vehicleFeeds_with_tripID['tripID'] == trip].sort_values(by = 'timestamp').reset_index(drop=True)
    try:
        stph_trips.route_gtfs_stops_mapper(feeds, output_html = trip + ".html")
    except:
        pass

Map saved as 350544507513318_inbound_trip_20250116_091447.html
Map saved as 350612076064508_outbound_trip_20250119_063733.html
Map saved as 350612076055969_outbound_trip_20250103_102933.html
Map saved as 350612079013932_inbound_trip_20250111_080704.html
Map saved as 350612076055969_outbound_trip_20241227_120351.html
Map saved as 350612076068145_outbound_trip_20250120_092939.html
Map saved as 350612076064508_inbound_trip_20250113_083149.html
Map saved as 350612076064508_outbound_trip_20250114_093419.html
Map saved as 350612076064508_outbound_trip_20250114_080854.html
Map saved as 350612076068145_outbound_trip_20250122_141443.html


In [82]:
for trip in sample_trips:
    feeds = all_vehicleFeeds_with_tripID[all_vehicleFeeds_with_tripID['tripID'] == trip].sort_values(by = 'timestamp').reset_index(drop=True)
    try:
        stph_trips.route_gtfs_stops_mapper(feeds, output_html = feeds['route'].values[0] + " - " + trip + ".html")
    except:
        pass

Map saved as Route 17 - 350544507513318_inbound_trip_20250116_091447.html
Map saved as Route 9 - 350612076064508_outbound_trip_20250119_063733.html
Map saved as Route 9 - 350612076055969_outbound_trip_20250103_102933.html
Map saved as Route 9 - 350612079013932_inbound_trip_20250111_080704.html
Map saved as Route 9 - 350612076055969_outbound_trip_20241227_120351.html
Map saved as Route 17 - 350612076068145_outbound_trip_20250120_092939.html
Map saved as Route 9 - 350612076064508_inbound_trip_20250113_083149.html
Map saved as Route 9 - 350612076064508_outbound_trip_20250114_093419.html
Map saved as Route 9 - 350612076064508_outbound_trip_20250114_080854.html
Map saved as Route 17 - 350612076068145_outbound_trip_20250122_141443.html


In [83]:
for imei in all_vehicleFeeds_with_tripID['imei'].unique().tolist():
    print(imei)
    imei_df = all_vehicleFeeds_with_tripID[all_vehicleFeeds_with_tripID['imei'] == imei]
    all_complete_trips = [trip for trip in imei_df['tripID'].unique().tolist() if 'cuttrip' not in trip]
    if len(all_complete_trips) > 0:
        if len(all_complete_trips) < 5:
            samp_size = len(all_complete_trips)
        else:
            samp_size = 5
        sample_trips = random.sample(all_complete_trips, samp_size)
        for trip in sample_trips:
            feeds = imei_df[imei_df['tripID'] == trip].sort_values(by = 'timestamp').reset_index(drop=True)
            try:
                stph_trips.route_gtfs_stops_mapper(feeds, output_html = feeds['route'].values[0] + " - " + trip + ".html")
            except:
                pass
    else:
        pass

350544507513318
Map saved as Route 17 - 350544507513318_inbound_trip_20250114_073602.html
Map saved as Route 17 - 350544507513318_inbound_trip_20250110_164817.html
Map saved as Route 17 - 350544507513318_inbound_trip_20250123_122253.html
Map saved as Route 17 - 350544507513318_outbound_trip_20250114_080214.html
Map saved as Route 17 - 350544507513318_outbound_trip_20250116_130555.html
350612076055969
Map saved as Route 9 - 350612076055969_outbound_trip_20250102_154727.html
Map saved as Route 9 - 350612076055969_outbound_trip_20250102_100022.html
Map saved as Route 9 - 350612076055969_inbound_trip_20250105_164654.html
Map saved as Route 9 - 350612076055969_inbound_trip_20241214_095903.html
Map saved as Route 9 - 350612076055969_outbound_trip_20241206_070106.html
350612076064508
Map saved as Route 9 - 350612076064508_inbound_trip_20250121_071617.html
Map saved as Route 9 - 350612076064508_inbound_trip_20250113_142258.html
Map saved as Route 9 - 350612076064508_outbound_trip_20250114_1825

<hr style="border:2px solid darkblue"> 

## Super summaries

In [77]:
tripSuperSummaries_list = []

for vehicle in all_vehicleFeeds_with_tripID['imei'].unique().tolist():
    print("Now processing: " + vehicle)
    my_df = all_vehicleFeeds_with_tripID[all_vehicleFeeds_with_tripID['imei'] == vehicle].reset_index(drop=True)
    trip_summaries = stph_trips.trip_super_summary(vehicle_feeds_with_tripID = my_df,
                                                   speed_cutoff = 5,
                                                   overwaiting_time = [90, 150],
                                                   overspeeding_thresholds = [60, 65],
                                                   harsh_acceleration = [(2.5, 2000), (2.5, 2500), (3.5, 2000), (3.5, 2500)],
                                                   harsh_braking = [(-2.5, -1000), (-3.5, -1000)])
    tripSuperSummaries_list.append(trip_summaries)

tripSuperSummaries = pd.concat(tripSuperSummaries_list, ignore_index = True)

Now processing: 350544507513318
Now processing: 350612076055969
Now processing: 350612076064508
Now processing: 350612076068145
Now processing: 350612076078946
Now processing: 350612079013932


In [78]:
tripSuperSummaries.head(3)

,Vehicle ID,Route,Trip type,Trip status,Date,Start time,End time,Total trip duration (min),Average speed (kph),Maximum speed (kph),...,Total overwaiting events (150),Average overwaiting time (150),"Total harsh acceleration events (2.5, 2000)","Total harsh acceleration events (2.5, 2500)","Total harsh acceleration events (3.5, 2000)","Total harsh acceleration events (3.5, 2500)","Total harsh braking events (-2.5, -1000)","Total harsh braking events (-3.5, -1000)",Total overspeeding duration (60-65 kph),Total overspeeding duration (65 kph)
0,350544507513318,Route 17,Inbound,Cut trip,"Jan 10, 2025",06:57 AM,07:01 AM,3.80,15.35,41.0,...,0,0.0,0,0,0,0,0,0,0.0,0.0
1,350544507513318,Route 17,Outbound,Complete trip,"Jan 10, 2025",07:18 AM,07:51 AM,33.62,9.05,36.0,...,1,171.0,0,0,0,0,0,0,0.0,0.0
2,350544507513318,Route 17,Inbound,Complete trip,"Jan 10, 2025",07:51 AM,08:15 AM,23.47,13.00,53.0,...,0,0.0,1,1,0,0,0,0,0.0,0.0


In [79]:
tripSuperSummaries.to_excel("Trip super summaries - Dec to Jan - GTFS amended 03112025.xlsx", index = False)